# <center> Code for phase construction, image alignment and compiling phase images into a video </center> 
### <center> By Dr. Joseph Vas </center>

The code is meant for aligning phase holography images and realigning with other for successive acquisition. The idea is to use hyperspy to calculate the amplitude and electron phase shift while passing through the sample. I'm using hyperspy for this. and using skimage for the re-alignment.

## modified for stack images

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import unravel_index

from skimage import data
from skimage.registration import phase_cross_correlation
from skimage.registration._phase_cross_correlation import _upsampled_dft
from scipy.ndimage import fourier_shift
import itertools
from skimage import io, filters, img_as_float
import warnings

import os

In [2]:
%pylab qt

Populating the interactive namespace from numpy and matplotlib


In [3]:
import hyperspy.api as hs

In [4]:
root_dir = '/Users/josephvas/Dropbox/check_figures/'

In [5]:
# Define a function to find all files with a specific extension
def find_files(root_dir, extension):
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith(extension):
                yield os.path.join(dirpath, filename)

In [ ]:
def holograph_reconstruction(TEM1):
    
    TEM1.set_signal_type('hologram')
    #if sideband_pos == 'none' and sideband_size == 'none':
    sideband_pos = TEM1.estimate_sideband_position(ap_cb_radius=None, sb='lower')
    sideband_size = TEM1.estimate_sideband_size(sideband_pos)
    wave_image = TEM1.reconstruct_phase(sb_size = sideband_size, sb_position =sideband_pos)
    wave_imge2 = wave_image.unwrapped_phase()

    return(np.abs(wave_image),wave_imge2)

In [23]:
# Call the function to find all .npy files
npy_files = find_files(root_dir, '.npy')

# Print out the file paths
for file_path in npy_files:
    TEM = hs.signals.Signal2D(np.load(file_path))
    
    print(file_path)


/Users/josephvas/Dropbox/check_figures/holo_state_01OL_value 25.npy
/Users/josephvas/Dropbox/check_figures/holo_state_00OL_value 25.npy
/Users/josephvas/Dropbox/check_figures/holo_state_02OL_value 25.npy
/Users/josephvas/Dropbox/check_figures/holo_state_04OL_value 25.npy
/Users/josephvas/Dropbox/check_figures/test2/holo_state_01OL_value 25.npy
/Users/josephvas/Dropbox/check_figures/test2/holo_state_00OL_value 25.npy
/Users/josephvas/Dropbox/check_figures/test2/holo_state_02OL_value 25.npy
/Users/josephvas/Dropbox/check_figures/test2/holo_state_04OL_value 25.npy
/Users/josephvas/Dropbox/check_figures/test2/check3/holo_state_01OL_value 25.npy
/Users/josephvas/Dropbox/check_figures/test2/check3/holo_state_00OL_value 25.npy
/Users/josephvas/Dropbox/check_figures/test2/check3/holo_state_02OL_value 25.npy
/Users/josephvas/Dropbox/check_figures/test2/check3/holo_state_04OL_value 25.npy


In [20]:
npy_files.throw

<function generator.throw>

In [ ]:
%%time

(absolute1, phase1) = holograph_reconstruction(TEM1)
(absolute2, phase2) = holograph_reconstruction(TEM2)
(absolute3, phase3) = holograph_reconstruction(TEM3)

phase1 = phase1/TEM1.data.shape[0]
phase2 = phase2/TEM2.data.shape[0]
phase3 = phase3/TEM3.data.shape[0]



In [ ]:
def realign_images (absolute1, phase1):
    
    #calculating the shift in the image using cross correlation
    
    shift = np.zeros([absolute1.data.shape[0]-1,2])

    #translation of images
    
    translated_image1 = np.zeros([absolute1.data.shape[1],absolute1.data.shape[2]])
    translated_image = np.zeros([absolute1.data.shape[1],absolute1.data.shape[2]])
    translated_phase1 = np.zeros([absolute1.data.shape[1],absolute1.data.shape[2]])
    translated_phase = np.zeros([absolute1.data.shape[1],absolute1.data.shape[2]])

    translated_image = absolute1.data[0,:,:]
    translated_phase = phase1.data[0,:,:]
    
    for k in range(0,absolute1.data.shape[0]-1):
        shift[k], error, diffphase = phase_cross_correlation(absolute1.data[0,:,:], absolute1.data[k+1,:,:], upsample_factor=100)
        for i,j in itertools.product(range(absolute1.data.shape[1]), range(absolute1.data.shape[2])):
            if i-int(shift[k,0])>0 and j-int(shift[k,1])>0 and i-int(shift[k,0])<absolute1.data.shape[1]  and j-int(shift[k,1])<absolute1.data.shape[2]:
                translated_image1[i,j] = absolute1.data[k,i-int(shift[k,0]),j-int(shift[k,1])] 
                translated_phase1[i,j] = phase2.data[k,i-int(shift[k,0]),j-int(shift[k,1])]

        translated_image = translated_image + translated_image1
        translated_phase = translated_phase + translated_phase1
        
    return(translated_image, translated_phase)



In [ ]:
(translated_image1, translated_phase1) = realign_images(absolute1, phase1)
(translated_image2, translated_phase2) = realign_images(absolute2, phase2)
(translated_image3, translated_phase3) = realign_images(absolute3, phase3)


In [ ]:
vmin_mul = -10
vmax_mul = 10

fig, ax = plt.subplots(nrows =2, ncols= 3)
image1 =translated_image1 
ax[0,0].set_title('holograph')
ax[0,0].imshow(image1, cmap='gray', \
                 vmin=vmin_mul*abs(np.mean(image1)), vmax=vmax_mul*abs(np.mean(image1)))

image2 =translated_image2
ax[0,1].set_title('holograph')
ax[0,1].imshow(image2, cmap='gray', \
                 vmin=vmin_mul*abs(np.mean(image2)), vmax=vmax_mul*abs(np.mean(image2)))

image5 =translated_image3
ax[0,2].set_title('holograph')
ax[0,2].imshow(image5, cmap='gray', \
                 vmin=vmin_mul*abs(np.mean(image5)), vmax=vmax_mul*abs(np.mean(image5)))


image3 =translated_phase1
ax[1,0].set_title('holograph')
ax[1,0].imshow(image3, cmap='gray', \
                 vmin=vmin_mul*abs(np.mean(image3)), vmax=vmax_mul*abs(np.mean(image3)))

image4 =translated_phase2
ax[1,1].set_title('holograph')
ax[1,1].imshow(image4, cmap='gray', \
                 vmin=vmin_mul*abs(np.mean(image4)), vmax=vmax_mul*abs(np.mean(image4)))

image6 =translated_phase2
ax[1,2].set_title('holograph')
ax[1,2].imshow(image6, cmap='gray', \
                 vmin=vmin_mul*abs(np.mean(image6)), vmax=vmax_mul*abs(np.mean(image6)))



In [ ]:
def gradients(image,pixel_size):
    dim = image.shape
    grad_X = np.zeros((dim[0],dim[1]))
    grad_Y = np.zeros((dim[0],dim[1]))
    for i in range (0,dim[0]-1):
        for j in range (0,dim[1]-1):
            grad_X[i,j] =(image[i,j+1]-image[i,j])/pixel_size
            grad_Y[i,j] =-(image[i+1,j]-image[i,j])/pixel_size
    
    return(grad_X,grad_Y)

In [ ]:
phi_crop = translated_phase1
pixel_dimensions = TEM1.original_metadata.ImageList.TagGroup0.ImageData.Calibrations.Dimension.TagGroup0.Scale*1e-9

h = 6.626e-34
electron_charge = 1.6e-19
t = 160e-9

coeff = h/(2*pi*electron_charge)
(grad_X,grad_Y) = gradients(phi_crop,pixel_dimensions)

Bx = coeff*grad_Y/t
By = -coeff*grad_X/t

In [ ]:
phi_crop = translated_phase2
pixel_dimensions = TEM1.original_metadata.ImageList.TagGroup0.ImageData.Calibrations.Dimension.TagGroup0.Scale*1e-9

h = 6.626e-34
electron_charge = 1.6e-19
t = 160e-9

coeff = h/(2*pi*electron_charge)
(grad_X,grad_Y) = gradients(phi_crop,pixel_dimensions)

Bx_1 = coeff*grad_Y/t
By_1 = -coeff*grad_X/t

In [ ]:
phi_crop = translated_phase3
pixel_dimensions = TEM1.original_metadata.ImageList.TagGroup0.ImageData.Calibrations.Dimension.TagGroup0.Scale*1e-9

h = 6.626e-34
electron_charge = 1.6e-19
t = 160e-9

coeff = h/(2*pi*electron_charge)
(grad_X,grad_Y) = gradients(phi_crop,pixel_dimensions)

Bx_2 = coeff*grad_Y/t
By_2 = -coeff*grad_X/t

In [ ]:
B = np.sqrt(abs(Bx**2+By**2))
B_phase = np.arctan2(By, Bx) * 180 / np.pi

B_max =np.max(B)
color =RGB_phase(B_phase, B, B_max)



In [ ]:
def RGB_phase(phase,mag,mag_max):
    color = np.zeros([phase.shape[0],phase.shape[1],4])
    phase = phase+180 # to change the range from [-180, 180] to [0, 360]
    for i in range(0, phase.shape[0]):
        for j in range(0,phase.shape[1]):
            #print(phase)
            if 0 <= phase[i,j] <= 120:
                r = (1/120)*phase[i,j]
                g = (-1/120)*phase[i,j]+1
                b = 0
            elif 120 <= phase[i,j] <= 240:
                r = (-1/120)*(phase[i,j]-120)+1
                g = 0
                b = (1/120)*(phase[i,j]- 120)
            elif 240 <= phase[i,j] <= 360:
                r = 0
                g = (1/120)*(phase[i,j]- 240)
                b = (-1/120)*(phase[i,j]-240)+1
            else:
                r = 0
                g = 0
                b = 0
            if r ==0 and g == 0 and b == 0:
                color[i,j,:] = [r,g,b,0]
            else:
                color[i,j,:] = [r,g,b,3*mag[i,j]/mag_max]
            #color[i,j,:] = [r,g,b,1]
            #color[i,j,:] = [r,g,b,mag[i,j]]
            #color[i,j,:] = [r,g,b,5*mag[i,j]/mag_max]
    return(color)

In [ ]:
fig, ax =plt.subplots(nrows=2, ncols =3)
vmn = -1
vmx = 1
vmn_1 = -1e-3
vmx_1 = 1e-3



ax[0,0].imshow(Bx,cmap ='gray', vmin = vmn, vmax = vmx)
ax[0,0].set_title('Bx')

ax[1,0].imshow(By,cmap ='gray', vmin = vmn, vmax = vmx)
ax[1,0].set_title('By')

ax[0,1].imshow(Bx_1,cmap ='gray', vmin = vmn, vmax = vmx)
ax[0,1].set_title('Bx')

ax[1,1].imshow(By_1,cmap ='gray', vmin = vmn, vmax = vmx)
ax[1,1].set_title('By')

ax[0,2].imshow(Bx_2,cmap ='gray', vmin = vmn, vmax = vmx)
ax[0,2].set_title('Bx')

ax[1,2].imshow(By_2,cmap ='gray', vmin = vmn, vmax = vmx)
ax[1,2].set_title('By')


In [ ]:
fig, ax =plt.subplots(nrows=2, ncols =3)
vmn = -.1
vmx = .1
vmn_1 = -1e-3
vmx_1 = 1e-3



ax[0,0].imshow(Bx-Bx_1,cmap ='gray', vmin = vmn, vmax = vmx)
ax[0,0].set_title('Bx-Bx_1')

ax[1,0].imshow(By-By_1,cmap ='gray', vmin = vmn, vmax = vmx)
ax[1,0].set_title('By-By_1')

ax[0,1].imshow(Bx-Bx_2,cmap ='gray', vmin = vmn, vmax = vmx)
ax[0,1].set_title('Bx-Bx_2')

ax[1,1].imshow(By-By_2,cmap ='gray', vmin = vmn, vmax = vmx)
ax[1,1].set_title('By-By_2')

ax[0,2].imshow(Bx_2-Bx_1,cmap ='gray', vmin = vmn, vmax = vmx)
ax[0,2].set_title('Bx_2-Bx_1')

ax[1,2].imshow(By_2-By_1,cmap ='gray', vmin = vmn, vmax = vmx)
ax[1,2].set_title('By_2-By_1')

In [ ]:
#angle = -5
#rotated_color = ndimage.rotate(color, angle)
#rotated_infocus = ndimage.rotate(in_focus.data,angle)
#rotated_Bx = ndimage.rotate(Bx, angle)
#rotated_By = ndimage.rotate(By, angle)
#rotated_diff = ndimage.rotate(under_focus_affined-in_focus_data,angle)

fig, ax =plt.subplots(nrows=2, ncols =2)
#xlim_min = 25
#xlim_max = 500
#ylim_min = 375
#ylim_max = 170
vmn = -20
vmx = 20

#ax[0,0].imshow(rotated_diff,cmap ='gray')
#ax[0,0].set_xlim([xlim_min,xlim_max])
#ax[0,0].set_ylim([ylim_min,ylim_max])
#ax[0,0].set_yticks([])
#ax[0,0].set_xticks([])
#ax[0,0].set_yticklabels([])
#ax[0,0].set_xticklabels([])
#ax[0,0].set_title('Difference',size =12)

ax[0,1].imshow(Bx,cmap ='gray', vmin = vmn, vmax = vmx)
#ax[0,1].set_xlim([xlim_min,xlim_max])
#ax[0,1].set_ylim([ylim_min,ylim_max])
#ax[0,1].set_yticks([])
#ax[0,1].set_xticks([])
#ax[0,1].set_yticklabels([])
#ax[0,1].set_xticklabels([])
ax[0,1].set_title('Bx')

ax[1,0].imshow(By,cmap ='gray', vmin = vmn, vmax = vmx)
#ax[1,0].set_xlim([xlim_min,xlim_max])
#ax[1,0].set_ylim([ylim_min,ylim_max])
#ax[1,0].set_yticks([])
#ax[1,0].set_xticks([])
#ax[1,0].set_yticklabels([])
#ax[1,0].set_xticklabels([])
ax[1,0].set_title('By')

ax[1,1].imshow(Bx, cmap ='gray')
ax[1,1].imshow(color)
#ax[1,1].set_xlim([xlim_min,xlim_max])
#ax[1,1].set_ylim([ylim_min,ylim_max])
#ax[1,1].set_yticks([])
#ax[1,1].set_xticks([])
#ax[1,1].set_yticklabels([])
#ax[1,1].set_xticklabels([])
ax[1,1].set_title('Magnetic field')

In [ ]:
color[0,0,:]

The next section is for removing linear background from phase

In [ ]:
translated_phase4 = remove_linear_background(translated_phase1)
translated_phase5 = remove_linear_background(translated_phase2)
translated_phase6 = remove_linear_background(translated_phase3)

In [ ]:
def remove_linear_background (phase):

    image = (phase - np.min(phase))/(np.max(phase)-np.min(phase))
    
    # Calculate the gradient of the image
    gradient = filters.rank.gradient(image, np.ones((3,3)))
    

    # Calculate the mean of the gradient along the x and y axes
    mean_gradient = [np.mean(gradient, axis=0), np.mean(gradient, axis=1)]

    # Create a meshgrid of the x and y coordinates of the image
    x, y = np.meshgrid(np.arange(image.shape[1]), np.arange(image.shape[0]))

    # Create a linear background image using the mean gradient
    background = mean_gradient[0]*x + mean_gradient[1]*y

    # Subtract the linear background from the image
    image_subtracted = image - background
    
    return(image_substracted)


In [ ]:
translated_phase4 = remove_linear_background(translated_phase1)

In [ ]:
phase = translated_phase2

In [ ]:
maximum = np.max(phase)
minimum = np.min(phase)
image = (phase - minimum)/(maximum-minimum)
    
    # Calculate the gradient of the image
grad_x, grad_y = np.gradient(image)
    
    # Create a meshgrid of the x and y coordinates of the image
x, y = np.meshgrid(np.arange(image.shape[1]), np.arange(image.shape[0]))

    # Create a linear background image using the mean gradient
background = np.mean(grad_x)*x + np.mean(grad_y)*y

    # Subtract the linear background from the image
image_subtracted = image - background
         
phase_withoutbackground = image_subtracted*(maximum-minimum)+minimum

In [ ]:
np.mean(grad_x)*1851

In [ ]:
(image[2,1]-image[0,1])/2

In [ ]:
x+y

In [ ]:
np.max(image-background)

In [ ]:
vmin_mul = -10
vmax_mul = 10

fig, ax = plt.subplots(nrows =1, ncols= 4)
image1 = image
ax[0].set_title('holograph')
ax[0].imshow(image1, cmap='gray', \
                 vmin=vmin_mul*abs(np.mean(image1)), vmax=vmax_mul*abs(np.mean(image1)))

image2 = background
ax[1].set_title('holograph')
ax[1].imshow(image2, cmap='gray', \
                 vmin=vmin_mul*abs(np.mean(image2)), vmax=vmax_mul*abs(np.mean(image2)))

image3 = image1 -image2
ax[2].set_title('holograph')
ax[2].imshow(image3, cmap='gray', \
                 vmin=vmin_mul*abs(np.mean(image3)), vmax=vmax_mul*abs(np.mean(image3)))


image4 = phase_withoutbackground
ax[3].set_title('holograph')
ax[3].imshow(image4, cmap='gray', \
                 vmin=vmin_mul*abs(np.mean(image4)), vmax=vmax_mul*abs(np.mean(image4)))
